# Systemy uczące się - Zad. dom. 1: Minimalizacja ryzyka empirycznego

### Autor rozwiązania: Joanna Cicha, 147963

Celem zadania jest zaimplementowanie własnego drzewa decyzyjnego wykorzystującego idee minimalizacji ryzyka empirycznego. 

## Twoja implementacja

Twoim celem jest uzupełnić poniższą klasę `TreeNode` tak by po wywołaniu `TreeNode.fit` tworzone było drzewo decyzyjne minimalizujące ryzyko empiryczne. Drzewo powinno wspierać problem klasyfikacji wieloklasowej (jak w przykładzie poniżej). Zaimplementowany algorytm nie musi (ale może) być analogiczny do zaprezentowanego na zajęciach algorytmu dla klasyfikacji. Wszelkie przejawy inwencji twórczej wskazane. Pozostaw komenatrze w kodzie, które wyjaśniają Twoje rozwiązanie.

Schemat oceniania:
- wynik na ukrytym zbiorze testowym (automatyczna ewaluacja) celność klasyfikacji >= prostego baseline'u 1 +20%,
- wynik na ukrytym zbiorze testowym (automatyczna ewaluacja) celność klasyfikacji >= prostego baseline'u 2 +40%,
- wynik na ukrytym zbiorze testowym (automatyczna ewaluacja) celność klasyfikacji >= bardziej zaawansowanego baseline'u 3 +40%.

Niedozwolone jest korzystanie z zewnętrznych bibliotek do tworzenia drzewa decyzyjnego (np. scikit-learn). 
Możesz jedynie korzystać z biblioteki numpy.

#### Uwaga: Możesz dowolnie modyfikować elementy tego notebooka (wstawiać komórki i zmieniać kod), o ile będzie się w nim na koniec znajdowała kompletna implementacja klasy `TreeNode` w jednej komórce.

In [65]:
import numpy as np

# Function to calculate entropy of a probability distribution
def H(p):
    if p in (0, 1):
        return 0.
    return -p * np.log2(p) - (1 - p) * np.log2(1 - p)

# Function to calculate entropy of a dataset
def entropy(array, n):
    counts = np.unique(array, return_counts=True)[1]
    s = np.sum(counts)
    e = 0
    for count in counts:
        e += H(count/ s)
    return len(array) / n * e

# Function to calculate Gini impurity of a dataset
def gini_impurity(array, n):
    counts = np.unique(array, return_counts=True)[1]
    s = np.sum(counts)
    impurity = 1 - np.sum((counts / s) ** 2)
    return len(array) / n * impurity

# Function to calculate information gain based on criterion (Gini or entropy)
def info_gain(zbior_1, zbior_2, criterion='gini'):
    H_func = gini_impurity if criterion == 'gini' else entropy # Gini gives better accuracy result than entropy
    if criterion == 'gini':
        H_func = gini_impurity
    else:
        H_func = entropy
    
    # Calculate overall, left, and right impurity
    n = len(zbior_1) + len(zbior_2)
    h = H_func(np.hstack([zbior_1, zbior_2]), n)
    h1 = H_func(zbior_1, n)
    h2 = H_func(zbior_2, n)

    # Calculate and return information gain
    h = h - (len(zbior_1) / n * h1 + len(zbior_2) / n * h2)
    return h

# Function to find unique splitting points for a feature
def get_splits(data, labels):
    indices = np.argsort(data)
    previous_cls = None
    values = []
    for value, cls in zip(data[indices], labels[indices]):
        if previous_cls is None:
            previous_cls = cls
            continue
        else:
            if cls != previous_cls:
                values.append(value)
    values = np.unique(values)
    return values

# Function to create a boolean mask for splitting a dataset
def split_mask(data, threshold):
    return data < threshold

# Function to find the best split for a dataset based on the specified criterion
def find_best_split(data, labels, criterion='gini'):
    igs = []
    for col in range(data.shape[1]):
        feature = data[:, col]
        splits = get_splits(feature, labels)
        for s in splits:
            mask = split_mask(feature, s)
            ig = info_gain(feature[mask], feature[~mask])
            igs.append((ig, col, s))
    if len(igs) < 1:
        return (0.0, None, None)
    return sorted(igs, reverse=True)[0]

# Definition of the TreeNode class for the decision tree
class TreeNode(object):
    def __init__(self, depth=0, max_depth=3, min_samples_split=2, min_samples_leaf=1):
        self.left = None # Typ: Node, wierzchołek znajdujący się po lewej stornie
        self.right = None # Typ: Node, wierzchołek znajdujący się po prawej stornie
        self.feature = None
        self.split = None
        self.depth = depth
        self.max_depth = max_depth
        self.min_samples_split = min_samples_split
        self.min_samples_leaf = min_samples_leaf
	
    def __format__(self, format_spec):
        return str(self)
    
    # Recursive function to fit the decision tree to the training data	
    def fit(self, data, target, max_depth = None):
        """
        Argumenty:
        data -- numpy.ndarray, macierz cech o wymiarach (n, m), gdzie n to liczba przykładów, a m to liczba cech
        target -- numpy.ndarray, wektor klas o długości n, gdzie n to liczba przykładów
        """
        
        # Find the best split for the current node
        ig, feature, split = find_best_split(data, target)
        if ig > 0. and len(data) >= self.min_samples_split and self.depth < self.max_depth:
            self.feature = feature
            self.split = split

            # Create masks for left and right branches based on the split
            mask = split_mask(data[:, feature], split)
            X_train_left, y_train_left = data[mask], target[mask]
            X_train_right, y_train_right = data[~mask], target[~mask]

            # Recursively fit the left and right branches
            self.left = TreeNode(depth=self.depth+1)
            self.right = TreeNode(depth=self.depth+1)
            self.left.fit(X_train_left, y_train_left, max_depth=max_depth)
            self.right.fit(X_train_right, y_train_right, max_depth=max_depth)
        else:
            # If conditions are not met, assign the most common class to the leaf node
            values, counts = np.unique(target, return_counts=True)
            self.cls = values[np.argmax(counts)]
	
    # Function to predict classes for new data points
    def predict(self, data):
        """
        Argumenty:
        data -- numpy.ndarray, macierz cech o wymiarach (n, m), gdzie n to liczba przykładów, a m to liczba cech

        Wartość zwracana:
        numpy.ndarray, wektor przewidzoanych klas o długości n, gdzie n to liczba przykładów
        """
        y_pred = np.zeros(data.shape[0])

        # Check if the node is a leaf
        if self.left is not None:
            feature = self.feature
            split = self.split
            mask = split_mask(data[:, feature], split)

            # Recursively predict for left and right branches
            left, right = data[mask], data[~mask]
            y_pred_left = self.left.predict(left)
            y_pred_right = self.right.predict(right)

            # Combine predictions for left and right branches
            y_pred[mask] = y_pred_left
            y_pred[~mask] = y_pred_right
        else:
            # If the node is a leaf, assign the class of the leaf to all instances
            y_pred[:] = self.cls
        return y_pred

## Przykład trenowanie i testowania drzewa
 
Później znajduje się przykład trenowania i testowania drzewa na zbiorze danych `iris`, który zawierający 150 próbek irysów, z czego każda próbka zawiera 4 atrybuty: długość i szerokość płatków oraz długość i szerokość działki kielicha. Każda próbka należy do jednej z trzech klas: `setosa`, `versicolor` lub `virginica`, które są zakodowane jak int.

Możesz go wykorzystać do testowania swojej implementacji. Możesz też zaimplementować własne testy lub użyć innych zbiorów danych, np. innych [zbiorów danych z scikit-learn](https://scikit-learn.org/stable/datasets/toy_dataset.html#toy-datasets).

In [62]:
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

data = load_iris()
X_train, X_test, y_train, y_test = train_test_split(data.data, data.target, test_size=0.33, random_state=2024)

tree_model = TreeNode()
tree_model.fit(X_train, y_train, max_depth = 3)
y_pred_train = tree_model.predict(X_train)
print(accuracy_score(y_train, y_pred_train))
y_pred = tree_model.predict(X_test)
print(accuracy_score(y_test, y_pred))

0.92
0.82


## Additional test on breast cancer dataset

In [66]:
tree_model_cancer = TreeNode(max_depth=5, min_samples_split=5, min_samples_leaf=2)
tree_model_cancer.fit(X_train_cancer, y_train_cancer)
y_pred_cancer = tree_model_cancer.predict(X_test_cancer)
print("Accuracy on Breast Cancer test set:", accuracy_score(y_test_cancer, y_pred_cancer))

Accuracy on Breast Cancer test set: 0.8138297872340425


## Additional test on wine dataset

In [72]:
from sklearn.datasets import load_wine
from sklearn.metrics import mean_squared_error

# Load the Wine dataset
wine = load_wine()
X, y = wine.data, wine.target

# Use a specific attribute as the target variable (e.g., flavanoids)
y = X[:, wine.feature_names.index('flavanoids')]

# Split the dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


# Instantiate and fit your decision tree for regression
tree_regressor = TreeNode()
tree_regressor.fit(X_train, y_train)

# Make predictions on the test set
y_pred = tree_regressor.predict(X_test)

# Evaluate mean squared error
mse = mean_squared_error(y_test, y_pred)
print(f"Mean Squared Error on the Wine Quality dataset: {mse:.2f}")


Mean Squared Error on the Wine Quality dataset: 0.87
